# <div align="center">Trabalho Computacional AV2: Modelos de Regressão e Classificação</div>

Equipe:

<ul>
    <li>Guilherme de Farias</li>
    <li>Tiago Farias</li>
</ul>

Bibliotecas utilizadas:

In [1]:
import pandas as pd
import numpy as np
import plotly.express as exp
import plotly.graph_objects as go

## <div align="center">Introdução</div>

O presente trabalho é composto por duas etapas em que deve-se utilizar os conceitos de IA baseados em modelos preditivos que realizam seu processo de aprendizagem através da minimização de uma função **custo (loss function)**. Em ambas etapas do trabalho, tais modelos utilizam o paradigma supervisionado para aprender a partir dos pares, vetor de características (variáveis regressoras) e variável dependente. Contudo, a tarefa da primeira etapa trata-se do desenvolvimento de um sistema que faz predições quantitativas (**regressão**), ao passo que a segunda etapa é caracterizada pelo desenvolvimento de um sistema que realiza predições qualitativas (**classificação**).

## <div align="center">Tarefa de Regressão. [3,0 pts]</div>

Para o problema de **regressão** solicita-se que faça o acesso ao conjunto de dados disponibilizado na plataforma AVA, chamado **aerogerador.dat**. A variável independente é uma medida de velocidade do vento, e a variável dependente é uma observação de potência gerada pelo aerogerador.

### Etapa 1

Faça uma visualização inicial dos dados através do **gráfico de espalhamento**. Nessa etapa, faça discussões sobre quais serão as características de um modelo que consegue entender o padrão entre variáveis regressoras e variáveis observadas.

In [2]:
df: pd.DataFrame = (
    pd.read_csv(
        "dados/aerogerador.dat", 
        sep=r'\s+|\t', # corresponder a uma ou mais ocorrências de espaços em branco (\s+) ou uma única tabulação (\t)
        engine='python', # a engine c, que é padrão, não suporta lidar com regex.
        header=None, 
        names=("velocidade do vento", "potência gerada")
    )
)

In [3]:
df.head(35)

,velocidade do vento,potência gerada
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0
5,0.0,0
6,0.0,0
7,0.0,0
8,0.0,0
9,0.0,0


In [ ]:
df.tail(35)

In [5]:
fig = exp.scatter(
    df, 
    x='velocidade do vento', 
    y='potência gerada',
    title="Velocidade do Vento x Potência Gerada",
    labels={
        'velocidade do vento': 
            'Velocidade do Vento (m/s)', 
        'potência gerada': 
            'Potência Gerada (W)'
    }
)

fig.show()

### Etapa 2

Em seguida, organize os dados de modo que as variáveis regressoras sejam armazenadas em uma matriz **(X)** de dimensão **ℝⁿˣᵖ**. Faça o mesmo para o vetor de variável dependente **(y)**, organizando em um vetor de dimensão **ℝⁿˣ¹**.

In [7]:
x: np.ndarray = np.array(df['velocidade do vento']) # Variável INdependente
y: np.ndarray = np.array(df['potência gerada']) # Variável dependente

# Reformatação para garantir que x e y sejam matrizes colunas
x = x.reshape(-1, 1) 
y = y.reshape(-1, 1) 
# -1 é a mesma coisa que len(x) ou len(y) nesses casos

# Adicionando uma coluna de 1s para o cálculo do intercepto (termo constante)
X: np.ndarray = np.concatenate(
    [np.ones((len(x), 1)), x], 
    axis=1
)

### Etapa 3

Os modelos a serem implementados nessa etapa serão: **MQO tradicional**, **MQO regularizado** (Tikhonov) e **Média de valores observáveis**. **Obs**: lembre-se que todos os modelos também estimam o valor do intercepto.

#### MQO tradicional

In [8]:
# Estimação dos coeficientes usando a fórmula do MMQO tradicional (beta = (X^T * X)^-1 * X^T * y)
B: np.ndarray = np.linalg.pinv(X.T @ X) @ X.T @ y

# Coeficientes da regressão
intercepto: float = B[0][0]
coeficiente_angular: float = B[1][0]

# Gerando pontos para a linha de regressão
x_pred: np.ndarray = np.linspace(min(x), max(x), 100)
X_pred: np.ndarray = np.concatenate([np.ones((len(x_pred), 1)), x_pred.reshape(-1, 1)], axis=1)
y_pred: np.ndarray = X_pred @ B

# Exibindo os coeficientes estimados
print(f"Intercepto (beta_0): {intercepto:.4f}")
print(f"Coeficiente angular (beta_1): {coeficiente_angular:.4f}")

Intercepto (beta_0): -217.6903
Coeficiente angular (beta_1): 56.4439


In [9]:
# Adicionando a linha de regressão ao gráfico
fig.add_trace(
    go.Scatter(
        x=x_pred.flatten(),
        y=y_pred.flatten(),
        mode='lines',
        name='Linha de Regressão',
        line={'color':'tan'}
    )
)

#### MQO regularizado (Tikhonov)